In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anand\\Desktop\\reume_projet\\AutoPrice-AI\\research'

In [3]:
os .chdir("../")

In [4]:
%pwd

'c:\\Users\\anand\\Desktop\\reume_projet\\AutoPrice-AI'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from mlProject.constants import*
from mlProject.utils.common import read_yaml, create_directories 


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

# This class is responsible for downloading and extracting the dataset
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [12]:
# Main script to execute the data ingestion process

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-13 05:05:21,699: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-13 05:05:21,701: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-13 05:05:21,702: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-13 05:05:21,703: INFO: common: created directory at: artifacts]
[2025-06-13 05:05:21,704: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-13 05:05:23,000: INFO: 1388048469: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 10308
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7929d1ef4f78146ca0839a49cd5ab5a1ef9e6073fbee50a03b666c1bbffa5eaa"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3ED0:FDD3C:1CBAB:8B61E:684B643B
Accept-Ranges: bytes
Date: Thu, 12 Ju